# Predcit test Scores of Students

## Task :
- Build an efficient ML model that predcits posttest scores with minimal error and high accuracy.

In [ ]:
import pandas as pd 
import matplotlib
import matplotlib.pyplot as plt 
import seaborn as sns 
import plotly.express as px
%matplotlib inline 

In [ ]:
sns.set_style('darkgrid')
matplotlib.rcParams['font.size']=14
matplotlib.rcParams['figure.figsize']=(10,6)
matplotlib.rcParams['figure.facecolor']='#00000000'

## Explarotary Analysis and Visualization

In [ ]:
scores_df=pd.read_csv('../input/predict-test-scores-of-students/test_scores.csv')

In [ ]:
scores_df

In [ ]:
scores_df.info()

In [ ]:
scores_df.describe()

## Pretest Score

In [ ]:
fig=px.histogram(scores_df,x='pretest',marginal='box',color='teaching_method',title='Distribution of Pretest score')
fig.update_layout(bargap=0.)
fig.show()

We can make the following observations-
- Students whose teaching method is experimental gained more marks in pretest as compared to students of standard teachimg method

## Perform preprocessing into categorical data

In [ ]:
scores_df

In [ ]:
scores_df.lunch.unique()

In [ ]:
lunch_type={'Does not qualify':0,
            'Qualifies for reduced/free lunch':1}
scores_df['lunch_code']=scores_df.lunch.map(lunch_type)

In [ ]:
scores_df

In [ ]:
teaching_type={'Standard':0,
            'Experimental':1}
scores_df['teaching_code']=scores_df.teaching_method.map(teaching_type)

In [ ]:
scores_df

In [ ]:
gender_type={'Female':0,
            'Male':1}
scores_df['gender_code']=scores_df.gender.map(gender_type)

In [ ]:
scores_df

In [ ]:
scores_df.school_type.unique()

In [ ]:
school_categories={'Non-public':0,
            'Public':1}
scores_df['school_type_code']=scores_df.school_type.map(school_categories)

In [ ]:
scores_df.school_setting.unique()

### Performing one hot encoding into School Setting

In [ ]:
from sklearn import preprocessing

In [ ]:
enc=preprocessing.OneHotEncoder()
enc.fit(scores_df[['school_setting']])
enc.categories_

In [ ]:
one_hot=enc.transform(scores_df[['school_setting']]).toarray()
scores_df[['Rural', 'Suburban', 'Urban']]=one_hot

In [ ]:
scores_df

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler=StandardScaler()



In [ ]:
scores_df.corr()

In [ ]:
sns.heatmap(scores_df.corr(),cmap='Reds',annot=True)
plt.title('Correaltion Matrix:')


In [ ]:
plt.title('Pretest vs. Posttest')
sns.scatterplot(data=scores_df,x='pretest',y='posttest',alpha=0.7,s=15);

In [ ]:
sns.regplot(data=scores_df,x='pretest',y='posttest')

In [ ]:
targets=scores_df['posttest']

In [ ]:
targets

### Loss/Cost function

In [ ]:
import numpy as np

In [ ]:
def rmse(targets,prediction):
    return np.sqrt(np.mean(np.square(targets - predictions)))


## Start creating model

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
# Creating inputs and targets
inputs=scores_df[['pretest','n_student','teaching_code','school_type_code','Rural','Suburban','Urban']]

#Create the model
model=LinearRegression()

model.fit(inputs,targets)

#model predict
predictions=model.predict(inputs)
#Loss :
loss=rmse(targets,predictions)
print('Loss :',loss)


In [ ]:
model.coef_

In [ ]:
inputs.columns

In [ ]:
weights_df=pd.DataFrame({
    'feature':np.append(inputs.columns,1),
    'weight':np.append(model.coef_,model.intercept_)

})
weights_df.sort_values('weight',ascending=False)

In [ ]:
standard_df=scores_df[scores_df.teaching_code==0]

# Creating inputs and targets
inputs=standard_df[['pretest','n_student','school_type_code','Rural','Suburban','Urban']]
targets=standard_df['posttest']
#Create the model
model=LinearRegression()

model.fit(inputs,targets)

#model predict
predictions=model.predict(inputs)
#Loss :
loss=rmse(targets,predictions)
print('Loss :',loss)

In [ ]:
weights_df=pd.DataFrame({
    'feature':np.append(inputs.columns,1),
    'weight':np.append(model.coef_,model.intercept_)

})
weights_df.sort_values('weight',ascending=False)

In [ ]:
experiment_df=scores_df[scores_df.teaching_code==1]

# Creating inputs and targets
inputs=experiment_df[['pretest','n_student','school_type_code','Rural','Suburban','Urban']]
targets=experiment_df['posttest']
#Create the model
model=LinearRegression()

model.fit(inputs,targets)

#model predict
predictions=model.predict(inputs)
#Loss :
loss=rmse(targets,predictions)
print('Loss :',loss)

In [ ]:
weights_df=pd.DataFrame({
    'feature':np.append(inputs.columns,1),
    'weight':np.append(model.coef_,model.intercept_)

})
weights_df.sort_values('weight',ascending=False)

In [ ]:
px.histogram(scores_df,x='teaching_method',color='school_type')

In [ ]:
scaler.fit(scores_df[['teaching_code']])
scaled_teaching_type_inputs=scaler.transform(scores_df[['teaching_code']])
scores_df['scaled_teaching']=scaled_teaching_type_inputs

In [ ]:
scores_df

In [ ]:
numeric_cols=['Urban','Suburban','Rural','n_student']
scaler=StandardScaler()
scaler.fit(scores_df[numeric_cols])

In [ ]:
scaled_inputs=scaler.transform(scores_df[numeric_cols])
scaled_inputs

In [ ]:
other_cols=['scaled_teaching','pretest','school_type_code']
other_data=scores_df[other_cols].values

In [ ]:
inputs=np.concatenate((scaled_inputs,other_data),axis=1)
targets=scores_df.posttest

model=LinearRegression().fit(inputs,targets)

predictions=model.predict(inputs)

loss=rmse(targets,predictions)
print('Loss :',loss)

r2_score=model.score(inputs,targets)
print("Accuracy of the model is :",r2_score*100)

In [ ]:
weights_df=pd.DataFrame({
    'feature':np.append(numeric_cols+other_cols,1),
    'weight':np.append(model.coef_,model.intercept_)
})
weights_df.sort_values('weight', ascending=False)
